In [1]:
import torch
import torch.optim as optim
from tqdm import tqdm
from milforvideo.model import FCL
from milforvideo.mil import MIL
from milforvideo.video import generate_dataloader
import numpy as np
from sklearn.metrics import roc_auc_score
import argparse
from milforvideo.video import VideoFeature
from typing import List

N_BATCH = 30
N_WORKER = 5
N_EPOCH = 5000
normal_path = "/workspace/MaskingUCSD/datasets/i3d/mask_background/UCSD_Anomaly_Dataset_v1p2_ped1_test_normal.pt"
anomalous_path = "/workspace/MaskingUCSD/datasets/i3d/mask_background/UCSD_Anomaly_Dataset_v1p2_ped1_test_anomalous.pt"

video_features_normal = torch.load(normal_path)
video_features_anomalous = torch.load(anomalous_path)

# Fit size of feature
model = FCL(input_size=video_features_normal[0].features[0].size(-1))

trainloader = generate_dataloader(
    video_features_normal,
    video_features_anomalous,
    batch_size=10,
    shuffle=True,
    num_workers=N_WORKER,
    V=13)

In [2]:
features_normal, features_anomalous, labels_normal, labels_anomalous = next(trainloader.__iter__())
features_normal

tensor([[[[0.0600, 0.0431, 0.2815,  ..., 0.0000, 0.0000, 0.1405],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]],

         [[0.0564, 0.0120, 0.2555,  ..., 0.0000, 0.0000, 0.1005],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]],

         [[0.0549, 0.0622, 0.3499,  ..., 0.0000, 0.0059, 0.0938],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]],

         ...,

         [[0.0324, 0.0448, 0.5165,  ..., 0.0000, 0.0000, 0.1104],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]],

         [[0.0556, 0.0770, 0.2915,  ..., 0.0000, 0.0483, 0.0980],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]],

         [[0.0964, 0.0346, 0.3286,  ..., 0.0000, 0.0144, 0.1009],
          [0.1886, 0.0991, 0.6295,  ..., 0.0146, 0.0000, 0.6614]]],


        [[[0.0266, 0.0148, 0.2252,  ..., 0.0000, 0.0000, 0.3963],
          [0.1187, 0.1244, 0.5511,  ..., 0.0000, 0.0000, 0.6384]],

         [[0.0250, 0.0286, 0.2417,  ..., 0.00

In [3]:
display(features_normal.size())
display(features_anomalous.size())
display(labels_normal.size())
display(labels_anomalous.size())

torch.Size([10, 13, 2, 1024])

torch.Size([10, 13, 2, 1024])

torch.Size([10, 13, 2])

torch.Size([10, 13, 2])

In [29]:
features_anomalous[(labels_anomalous > -1).any(dim=1).any(dim=1), :, 1]

tensor([[[9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01],
         [9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01],
         [9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01],
         ...,
         [9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01],
         [9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01],
         [9.5787e-02, 6.8562e-02, 6.1071e-01,  ..., 0.0000e+00,
          6.6291e-05, 7.5324e-01]],

        [[1.2025e-01, 2.9083e-02, 4.7285e-01,  ..., 0.0000e+00,
          0.0000e+00, 5.8625e-01],
         [1.2025e-01, 2.9083e-02, 4.7285e-01,  ..., 0.0000e+00,
          0.0000e+00, 5.8625e-01],
         [1.2025e-01, 2.9083e-02, 4.7285e-01,  ..., 0.0000e+00,
          0.0000e+00, 5.8625e-01],
         ...,
         [5.8879e-02, 2.0490e-02, 3.5503e-01,  ..., 0.0000e+00,
          2.509

In [27]:
labels_anomalous

tensor([[[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 0, -1]],

        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0]],

        [[ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 0, -1],
         [ 0, -1]],

        [[ 0, -1],
         [ 0, -1],
         [ 0, -1],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 1,  0],
         [ 0, -1],
         [ 0, -1]],

    